COVID 19 Analysis

,<font size="4">Section 1: Data Preperation</font>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# Loading datasets required for analysis

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")


In [7]:
def urban_to_float(x):
    try:
        return float(x.strip('%'))/100
    except:
        return np.nan

def cast_age_to_int(x):
    try:
        return int(x)
    except:
        return np.nan

def cast_fertility_to_float(x):
    try:
        return float(x)
    except:
        return np.nan


population_data = pd.read_csv("population_by_country_2020.csv", converters={'Urban Pop %':urban_to_float,
                                                                                                             'Fert. Rate':cast_fertility_to_float,
                                                                                                             'Med. Age':cast_age_to_int})
population_data.rename(columns={'Country (or dependency)': 'country',
                             'Population (2020)' : 'population',
                             'Density (P/Km²)' : 'density',
                             'Fert. Rate' : 'fertility',
                             'Med. Age' : "age",
                             'Urban Pop %' : 'urban_percentage'}, inplace=True)

population_data['country'] = population_data['country'].replace('United States', 'US')
population_data['country'] = population_data['country'].replace('United Kingdom', 'UK')
population_data['country'] =population_data['country'].str.lower()
population_data = population_data[["country", "population", "density", "fertility", "age", "urban_percentage"]]

population_data

,country,population,density,fertility,age,urban_percentage
0,china,1438207241,153,1.7,38.0,0.61
1,india,1377233523,464,2.2,28.0,0.35
2,us,330610570,36,1.8,38.0,0.83
3,indonesia,272931713,151,2.3,30.0,0.56
4,pakistan,219992900,287,3.6,23.0,0.35
...,...,...,...,...,...,...
230,montserrat,4991,50,NaN,NaN,0.10
231,falkland islands,3458,0,NaN,NaN,0.66
232,niue,1624,6,NaN,NaN,0.46
233,tokelau,1354,136,NaN,NaN,0.00


In [8]:
covid_data = pd.read_csv('covid_19_clean_complete.csv', parse_dates=['Date'])

covid_data.rename(columns={'ObservationDate': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Last Update':'last_updated',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)

cases = ['confirmed', 'deaths', 'recovered', 'active']

#No of Active Case computed as ,active = confirmed - deaths - recovered
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

# country name handling
covid_data['country'] = covid_data['country'].replace('Mainland China', 'China')
covid_data['country'] = covid_data['country'].replace('United States', 'US')
covid_data['country'] = covid_data['country'].replace( 'United Kingdom','UK')
covid_data['country'] = covid_data['country'].str.lower()
# filling missing values 
covid_data[['state']] = covid_data[['state']].fillna('')
covid_data[cases] = covid_data[cases].fillna(0)
covid_data.rename(columns={'Date':'date'}, inplace=True)

covid_data

,state,country,Lat,Long,date,confirmed,deaths,recovered,active
0,,afghanistan,33.000000,65.000000,2020-01-22,0,0,0,0
1,,albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
24361,Saint Pierre and Miquelon,france,46.885200,-56.315900,2020-04-23,1,0,0,1
24362,,south sudan,6.877000,31.307000,2020-04-23,5,0,0,5
24363,,western sahara,24.215500,-12.885800,2020-04-23,6,0,5,1
24364,,sao tome and principe,0.186360,6.613081,2020-04-23,4,0,0,4


In [9]:
data = pd.merge(covid_data, population_data, on='country')
data

,state,country,Lat,Long,date,confirmed,deaths,recovered,active,population,density,fertility,age,urban_percentage
0,,afghanistan,33.000000,65.000000,2020-01-22,0,0,0,0,38742911,60,4.6,18.0,0.25
1,,afghanistan,33.000000,65.000000,2020-01-23,0,0,0,0,38742911,60,4.6,18.0,0.25
2,,afghanistan,33.000000,65.000000,2020-01-24,0,0,0,0,38742911,60,4.6,18.0,0.25
3,,afghanistan,33.000000,65.000000,2020-01-25,0,0,0,0,38742911,60,4.6,18.0,0.25
4,,afghanistan,33.000000,65.000000,2020-01-26,0,0,0,0,38742911,60,4.6,18.0,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23152,,yemen,15.552727,48.516388,2020-04-19,1,0,0,1,29687214,56,3.8,20.0,0.38
23153,,yemen,15.552727,48.516388,2020-04-20,1,0,0,1,29687214,56,3.8,20.0,0.38
23154,,yemen,15.552727,48.516388,2020-04-21,1,0,0,1,29687214,56,3.8,20.0,0.38
23155,,yemen,15.552727,48.516388,2020-04-22,1,0,0,1,29687214,56,3.8,20.0,0.38


Step 2: Preparing the data

In [10]:
temperature_data = pd.read_csv("temperature_dataframe.csv")
temperature_data['country'] = temperature_data['country'].replace('USA', 'US')
#df_temperature['country'] = df_temperature['country'].replace('UK', 'United Kingdom')
temperature_data['country'] = temperature_data['country'].str.lower()
temperature_data = temperature_data[["country", "province", "date", "humidity", "sunHour", "tempC", "windspeedKmph"]].reset_index()
temperature_data.rename(columns={'province': 'state'}, inplace=True)
temperature_data["date"] = pd.to_datetime(temperature_data['date'])
temperature_data['state'] = temperature_data['state'].fillna('')
# df_temperature.info()

In [11]:
data = pd.merge(data,temperature_data, on=['country','state','date'])
data['mortality_rate'] = data['deaths'] / data['confirmed']

In [12]:
data

,state,country,Lat,Long,date,confirmed,deaths,recovered,active,population,density,fertility,age,urban_percentage,index,humidity,sunHour,tempC,windspeedKmph,mortality_rate
0,,afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0,38742911,60,4.6,18.0,0.25,0,65.0,8.7,-1.0,8.0,NaN
1,,afghanistan,33.0000,65.0000,2020-01-23,0,0,0,0,38742911,60,4.6,18.0,0.25,1,59.0,8.7,-3.0,8.0,NaN
2,,afghanistan,33.0000,65.0000,2020-01-24,0,0,0,0,38742911,60,4.6,18.0,0.25,2,71.0,7.1,0.0,7.0,NaN
3,,afghanistan,33.0000,65.0000,2020-01-25,0,0,0,0,38742911,60,4.6,18.0,0.25,3,79.0,8.7,0.0,7.0,NaN
4,,afghanistan,33.0000,65.0000,2020-01-26,0,0,0,0,38742911,60,4.6,18.0,0.25,4,64.0,8.7,-1.0,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,,zambia,-15.4167,28.2833,2020-03-17,0,0,0,0,18273379,25,4.7,18.0,0.45,16672,86.0,6.0,24.0,12.0,NaN
11634,,zambia,-15.4167,28.2833,2020-03-18,2,0,0,2,18273379,25,4.7,18.0,0.45,16673,84.0,8.8,25.0,12.0,0.0
11635,,zambia,-15.4167,28.2833,2020-03-19,2,0,0,2,18273379,25,4.7,18.0,0.45,16674,86.0,8.8,25.0,10.0,0.0
11636,,zambia,-15.4167,28.2833,2020-03-20,2,0,0,2,18273379,25,4.7,18.0,0.45,16675,81.0,11.6,25.0,11.0,0.0


In [8]:
#data.to_csv("coolcovid.csv")

In [13]:
data=data.drop(['index'], axis = 1) 

In [14]:
data=data.drop_duplicates()

In [15]:
data

,state,country,Lat,Long,date,confirmed,deaths,recovered,active,population,density,fertility,age,urban_percentage,humidity,sunHour,tempC,windspeedKmph,mortality_rate
0,,afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0,38742911,60,4.6,18.0,0.25,65.0,8.7,-1.0,8.0,NaN
1,,afghanistan,33.0000,65.0000,2020-01-23,0,0,0,0,38742911,60,4.6,18.0,0.25,59.0,8.7,-3.0,8.0,NaN
2,,afghanistan,33.0000,65.0000,2020-01-24,0,0,0,0,38742911,60,4.6,18.0,0.25,71.0,7.1,0.0,7.0,NaN
3,,afghanistan,33.0000,65.0000,2020-01-25,0,0,0,0,38742911,60,4.6,18.0,0.25,79.0,8.7,0.0,7.0,NaN
4,,afghanistan,33.0000,65.0000,2020-01-26,0,0,0,0,38742911,60,4.6,18.0,0.25,64.0,8.7,-1.0,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,,zambia,-15.4167,28.2833,2020-03-17,0,0,0,0,18273379,25,4.7,18.0,0.45,86.0,6.0,24.0,12.0,NaN
11634,,zambia,-15.4167,28.2833,2020-03-18,2,0,0,2,18273379,25,4.7,18.0,0.45,84.0,8.8,25.0,12.0,0.0
11635,,zambia,-15.4167,28.2833,2020-03-19,2,0,0,2,18273379,25,4.7,18.0,0.45,86.0,8.8,25.0,10.0,0.0
11636,,zambia,-15.4167,28.2833,2020-03-20,2,0,0,2,18273379,25,4.7,18.0,0.45,81.0,11.6,25.0,11.0,0.0


In [12]:
#data.to_csv("coolcovid1.csv")

Creating a consolidated table , which gives the country wise total defined cases

In [16]:
temp = data.groupby(['country', 'state'])['confirmed', 'deaths', 'recovered', 'active'].max()

In [17]:
data

,state,country,Lat,Long,date,confirmed,deaths,recovered,active,population,density,fertility,age,urban_percentage,humidity,sunHour,tempC,windspeedKmph,mortality_rate
0,,afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0,38742911,60,4.6,18.0,0.25,65.0,8.7,-1.0,8.0,NaN
1,,afghanistan,33.0000,65.0000,2020-01-23,0,0,0,0,38742911,60,4.6,18.0,0.25,59.0,8.7,-3.0,8.0,NaN
2,,afghanistan,33.0000,65.0000,2020-01-24,0,0,0,0,38742911,60,4.6,18.0,0.25,71.0,7.1,0.0,7.0,NaN
3,,afghanistan,33.0000,65.0000,2020-01-25,0,0,0,0,38742911,60,4.6,18.0,0.25,79.0,8.7,0.0,7.0,NaN
4,,afghanistan,33.0000,65.0000,2020-01-26,0,0,0,0,38742911,60,4.6,18.0,0.25,64.0,8.7,-1.0,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,,zambia,-15.4167,28.2833,2020-03-17,0,0,0,0,18273379,25,4.7,18.0,0.45,86.0,6.0,24.0,12.0,NaN
11634,,zambia,-15.4167,28.2833,2020-03-18,2,0,0,2,18273379,25,4.7,18.0,0.45,84.0,8.8,25.0,12.0,0.0
11635,,zambia,-15.4167,28.2833,2020-03-19,2,0,0,2,18273379,25,4.7,18.0,0.45,86.0,8.8,25.0,10.0,0.0
11636,,zambia,-15.4167,28.2833,2020-03-20,2,0,0,2,18273379,25,4.7,18.0,0.45,81.0,11.6,25.0,11.0,0.0


In [19]:
temp = data.groupby('date')['confirmed', 'deaths', 'recovered', 'active'].sum().reset_index()
temp = temp[temp['date']==max(temp['date'])].reset_index(drop=True)
temp.style.background_gradient(cmap='Pastel1')

,date,confirmed,deaths,recovered,active
0,2020-03-21 00:00:00,243174,11582,87251,144341


In [ ]:
temp_f = data.sort_values(by='confirmed', ascending=False)
temp_f = temp_f.reset_index(drop=True)
temp_f.style.background_gradient(cmap='Reds')

**Finally, this is how the table looks like and our analysis is baselined with this table**

In [18]:
temp_f.head(10)

,state,country,Lat,Long,date,confirmed,deaths,recovered,active,population,density,fertility,age,urban_percentage,humidity,sunHour,tempC,windspeedKmph,mortality_rate,Active
0,Hubei,china,30.9756,112.2707,2020-03-19,67800,3130,57682,6988,1438207241,153,1.7,38.0,0.61,13.0,11.6,18.0,18.0,0.046165,6988
1,Hubei,china,30.9756,112.2707,2020-03-18,67800,3122,56927,7751,1438207241,153,1.7,38.0,0.61,19.0,8.8,24.0,16.0,0.046047,7751
2,Hubei,china,30.9756,112.2707,2020-03-20,67800,3133,58382,6285,1438207241,153,1.7,38.0,0.61,17.0,11.6,24.0,9.0,0.046209,6285
3,Hubei,china,30.9756,112.2707,2020-03-21,67800,3139,58946,5715,1438207241,153,1.7,38.0,0.61,23.0,11.6,20.0,7.0,0.046298,5715
4,Hubei,china,30.9756,112.2707,2020-03-17,67799,3111,56003,8685,1438207241,153,1.7,38.0,0.61,20.0,11.6,22.0,8.0,0.045886,8685
5,Hubei,china,30.9756,112.2707,2020-03-16,67798,3099,55142,9557,1438207241,153,1.7,38.0,0.61,36.0,11.6,18.0,7.0,0.045709,9557
6,Hubei,china,30.9756,112.2707,2020-03-15,67794,3085,54288,10421,1438207241,153,1.7,38.0,0.61,28.0,11.6,18.0,9.0,0.045506,10421
7,Hubei,china,30.9756,112.2707,2020-03-14,67790,3075,52960,11755,1438207241,153,1.7,38.0,0.61,17.0,11.6,15.0,15.0,0.045361,11755
8,Hubei,china,30.9756,112.2707,2020-03-13,67786,3062,51553,13171,1438207241,153,1.7,38.0,0.61,13.0,11.6,13.0,10.0,0.045172,13171
9,Hubei,china,30.9756,112.2707,2020-03-12,67781,3056,50318,14407,1438207241,153,1.7,38.0,0.61,20.0,11.6,13.0,15.0,0.045086,14407
